# 🚀 DeepSeek-R1 Model with Azure AI Inference 🧠

**DeepSeek-R1** is a state-of-the-art reasoning model combining reinforcement learning and supervised fine-tuning, excelling at complex reasoning tasks with 37B active parameters and 128K context window.

In this notebook, you'll learn to:
1. **Initialize** the ChatCompletionsClient for Azure serverless endpoints
2. **Chat** with DeepSeek-R1 using reasoning extraction
3. **Implement** a travel planning example with step-by-step reasoning
4. **Leverage** the 128K context window for complex scenarios

## Why DeepSeek-R1?
- **Advanced Reasoning**: Specializes in chain-of-thought problem solving
- **Massive Context**: 128K token window for detailed analysis
- **Efficient Architecture**: 37B active parameters from 671B total
- **Safety Integrated**: Built-in content filtering capabilities


## 1. Setup & Authentication

Required packages:
- `azure-ai-inference`: For chat completions
- `python-dotenv`: For environment variables

.env file requirements:
```bash
AZURE_INFERENCE_ENDPOINT=<your-endpoint-url>
AZURE_INFERENCE_KEY=<your-api-key>
MODEL_NAME=DeepSeek-R1
```

In [1]:
import os
import re
from dotenv import load_dotenv
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from pathlib import Path

# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent.parent
load_dotenv(parent_dir / '.env')

# Load environment
load_dotenv()
endpoint = os.getenv("AZURE_INFERENCE_ENDPOINT")
key = os.getenv("AZURE_INFERENCE_KEY")
model_name = os.getenv("MODEL_NAME")

# Initialize client
try:
    client = ChatCompletionsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(key)
    )
    print("✅ Client initialized | Model:", model_name)
except Exception as e:
    print("❌ Initialization failed:", e)

✅ Client initialized | Model: DeepSeek-R1


## 2. Intelligent Travel Planning ✈️

Demonstrate DeepSeek-R1's reasoning capabilities for trip planning:

In [2]:
def plan_trip_with_reasoning(query, show_thinking=False):
    """Get travel recommendations with reasoning extraction"""
    messages = [
        SystemMessage(content="You are a travel expert. Provide detailed plans with rationale."),
        UserMessage(content=f"{query} Include hidden gems and safety considerations.")
    ]
    
    response = client.complete(
        messages=messages,
        model=model_name,
        temperature=0.7,
        max_tokens=1024
    )
    
    content = response.choices[0].message.content
    
    # Extract reasoning if present
    if show_thinking:
        match = re.search(r"<think>(.*?)</think>(.*)", content, re.DOTALL)
        if match:
            return {"thinking": match.group(1).strip(), "answer": match.group(2).strip()}
    return content

# Example usage
query = "Plan a 5-day cultural trip to Kyoto in April"
result = plan_trip_with_reasoning(query, show_thinking=True)

print("🗺️ Query:", query)
if isinstance(result, dict):
    print("\n🧠 Thinking Process:", result["thinking"])
    print("\n📝 Final Answer:", result["answer"])
else:
    print("\n📝 Response:", result)

🗺️ Query: Plan a 5-day cultural trip to Kyoto in April

🧠 Thinking Process: Okay, the user wants a 5-day cultural trip to Kyoto in April, including hidden gems and safety tips. Let me start by recalling Kyoto's main attractions and the best times to visit them. April is cherry blossom season, so some spots might be crowded. They mentioned hidden gems, so I need to include less touristy places. Also, safety considerations are important, especially for travelers.

First day: Arrival and initial exploration. Maybe start with Fushimi Inari Taisha since it's iconic. But the user wants hidden gems, so perhaps suggest going early to avoid crowds and explore the lesser-known trails there. Then maybe a traditional tea house in Gion, but not the main area. Safety tip: be cautious with taxis, use public transport.

Second day: Focus on temples and gardens. Kinkaku-ji is a must, but it's crowded. Suggest Ryoan-ji nearby, which is a Zen garden and maybe less crowded. Then maybe a hidden temple like

## 3. Technical Problem Solving 💻

Showcase coding/optimization capabilities:

In [3]:
def solve_technical_problem(problem):
    """Solve complex technical problems with structured reasoning"""
    response = client.complete(
        messages=[
            UserMessage(content=f"{problem} Please reason step by step, and put your final answer within \boxed{{}}.")
        ],
        model=model_name,
        temperature=0.3,
        max_tokens=2048
    )
    
    return response.choices[0].message.content

# Database optimization example
problem = """How can I optimize a PostgreSQL database handling 10k transactions/second?
Consider indexing strategies, hardware requirements, and query optimization."""

print("🔧 Problem:", problem)
print("\n⚙️ Solution:", solve_technical_problem(problem))

🔧 Problem: How can I optimize a PostgreSQL database handling 10k transactions/second?
Consider indexing strategies, hardware requirements, and query optimization.

⚙️ Solution: <think>
Okay, so I need to figure out how to optimize a PostgreSQL database that's handling 10k transactions per second. That's a pretty high load, so there's a lot to consider here. Let me start by breaking down the problem into parts. The user mentioned indexing strategies, hardware requirements, and query optimization. Maybe I should tackle each of these areas one by one.

First, indexing. I know that indexes are crucial for speeding up queries, but they can also slow down writes because every insert, update, or delete has to update the index. So, for a high transaction rate, I need to make sure that indexes are optimized. Maybe using the right types of indexes. For example, B-tree indexes are the default in PostgreSQL and are good for a range of queries. But if there are a lot of columns used in WHERE clause

## 4. Best Practices & Considerations

1. **Reasoning Handling**: Use regex to separate <think> content from final answers
2. **Safety**: Built-in content filtering - handle HttpResponseError for violations
3. **Performance**:
   - Max tokens: 4096
   - Rate limit: 200K tokens/minute
4. **Cost**: Pay-as-you-go with serverless deployment
5. **Streaming**: Implement response streaming for long completions

```python
# Streaming example
response = client.complete(..., stream=True)
for chunk in response:
    print(chunk.choices[0].delta.content or "", end="")
```

## 🎯 Key Takeaways
- Leverage 128K context for detailed analysis
- Extract reasoning steps for debugging/analysis
- Combine with Azure AI Content Safety for production
- Monitor token usage via response.usage

> Always validate model outputs for critical applications!